# Train Equalized Div-Free Noise DDPM on Colab

Trains the `fwd_diff_eq_divfree` experiment — spectrally-equalized divergence-free noise
that fixes the low-frequency spectral gap in standard div-free noise.

**Training improvements (Feb 2026):**
- **Cosine LR schedule** with linear warmup (breaks through Adam plateau)
- **AdamW** with weight decay (better generalization)
- **EMA** of model weights (smoother inference, standard in DDPM)
- **Velocity-aware augmentation** (H/V flips preserving ∇·v=0)
- **Full test-set evaluation** (all 1965 samples, not just 320)

**Prerequisites:**
1. Push latest code to GitHub (including the equalized noise class)
2. Upload `stjohn_hourly_5m_velocity_ramhead_v2.mat` (942 MB) to Google Drive
   - Put it in: `My Drive/Ocean Inpainting/`
3. Upload `boundaries.yaml` to the same Drive folder

**Runtime:** Select GPU runtime (Runtime → Change runtime type → T4 GPU)

**Estimated time:** ~15-20 hours for 1000 epochs on T4

## 1. Setup: Mount Drive, Clone Repo, Install Deps

In [ ]:
# Mount Google Drive (for data + saving checkpoints)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone the repo
import os
REPO_URL = 'https://github.com/DrCaley/difussionInpaintingVectorFields.git'
REPO_DIR = '/content/diffusionInpaintingVectorFields'

if os.path.exists(REPO_DIR):
    print(f'Repo already cloned at {REPO_DIR}, pulling latest...')
    !cd {REPO_DIR} && git pull
else:
    !git clone {REPO_URL} {REPO_DIR}

%cd {REPO_DIR}
!pwd

In [ ]:
# Install dependencies (Colab already has torch, numpy, matplotlib)
!pip install -q tqdm pyyaml scipy gpytorch

In [ ]:
# Symlink data from Google Drive into the expected location
DRIVE_DATA = '/content/drive/MyDrive/Ocean Inpainting'
LOCAL_DATA = f'{REPO_DIR}/data/rams_head'

os.makedirs(LOCAL_DATA, exist_ok=True)

# Symlink the .mat file (942 MB — don't copy, just link)
mat_src = f'{DRIVE_DATA}/stjohn_hourly_5m_velocity_ramhead_v2.mat'
mat_dst = f'{LOCAL_DATA}/stjohn_hourly_5m_velocity_ramhead_v2.mat'
bounds_src = f'{DRIVE_DATA}/boundaries.yaml'
bounds_dst = f'{LOCAL_DATA}/boundaries.yaml'

for src, dst in [(mat_src, mat_dst), (bounds_src, bounds_dst)]:
    if not os.path.exists(dst):
        assert os.path.exists(src), f'Missing: {src}\nUpload to Google Drive first!'
        os.symlink(src, dst)
        print(f'Linked {dst} → {src}')
    else:
        print(f'Already exists: {dst}')

!ls -lh {LOCAL_DATA}/

In [ ]:
# Generate the data.pickle file (train/val/test split from raw .mat)
# This is required by DDInitializer and only needs to run once per Colab session
import os
PICKLE_PATH = f'{REPO_DIR}/data.pickle'

if not os.path.exists(PICKLE_PATH):
    print('Generating data.pickle from .mat file...')
    %cd {REPO_DIR}
    !python data_prep/spliting_data_sets.py
    assert os.path.exists(PICKLE_PATH), 'data.pickle was not created!'
    print(f'Created: {PICKLE_PATH}')
else:
    print(f'data.pickle already exists: {PICKLE_PATH}')

!ls -lh {PICKLE_PATH}

In [ ]:
# Verify GPU is available
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('WARNING: No GPU! Go to Runtime → Change runtime type → T4 GPU')

## 2. Validate Experiment Config

In [ ]:
# Dry-run to validate config
!PYTHONPATH=. python experiments/run_experiment.py \
    --dry-run experiments/01_noise_strategy/fwd_divfree_equalized/config.yaml

## 3. Quick Smoke Test (3 epochs)

In [ ]:
# Smoke test — verify everything works on GPU before committing to full run
!PYTHONPATH=. python experiments/run_experiment.py \
    --smoke experiments/01_noise_strategy/fwd_divfree_equalized/config.yaml

## 4. Create Colab-Optimized Config & Train (1000 epochs)

The experiment config uses local defaults. This cell writes a Colab-specific
override with GPU batch size, LR scaling, and all training improvements.

**Important:** Colab may disconnect after ~4-12 hours depending on your plan.
The training saves best checkpoint automatically, so you can resume if interrupted.

Tips to avoid disconnection:
- Keep the browser tab open and active
- Colab Pro gives longer runtime (~24h)
- Checkpoints are saved to the experiment results/ folder

In [ ]:
# ── Adjust batch size for your GPU ──
COLAB_BATCH_SIZE = 64   # 64 for T4/V100, 128 for A100
BASE_LR = 0.0003
BASE_BS = 16
COLAB_LR = BASE_LR * (COLAB_BATCH_SIZE / BASE_BS)

# Write Colab-optimized config with training improvements
COLAB_CFG = f'{REPO_DIR}/experiments/01_noise_strategy/fwd_divfree_equalized/colab_config.yaml'

with open(COLAB_CFG, 'w') as f:
    f.write(f"""# Auto-generated Colab config — GPU-optimized with training improvements
model_name: fwd_diff_eq_divfree_eps_t250
noise_function: fwd_diff_eq_divfree
unet_type: standard
prediction_target: eps
mask_xt: false
p_uncond: 0.0
batch_size: {COLAB_BATCH_SIZE}
lr: {COLAB_LR}
max_grad_norm: 1.0

# ── Training improvements ──
weight_decay: 0.0001       # AdamW regularization
lr_schedule: cosine        # cosine annealing with warmup
warmup_epochs: 10          # linear warmup from lr*0.001 → lr
use_ema: true              # exponential moving average of weights
ema_decay: 0.9999          # standard DDPM EMA decay
augment: false             # disabled — site-specific ocean data shouldn't be flipped
""")

print(f'Wrote Colab config: batch_size={COLAB_BATCH_SIZE}, lr={COLAB_LR}')
!cat {COLAB_CFG}

# Full training run
!PYTHONPATH=. python experiments/run_experiment.py {COLAB_CFG}

## 5. Save Results to Google Drive

Copy checkpoints to Drive so they survive Colab shutdown.

In [ ]:
import shutil

SRC = f'{REPO_DIR}/experiments/01_noise_strategy/fwd_divfree_equalized/results'
DST = '/content/drive/MyDrive/Ocean Inpainting/training_results/fwd_divfree_equalized'

os.makedirs(DST, exist_ok=True)

# Copy all checkpoint and log files
copied = 0
for f in os.listdir(SRC):
    if f.endswith(('.pt', '.yaml', '.csv', '.png')):
        shutil.copy2(os.path.join(SRC, f), os.path.join(DST, f))
        print(f'  Copied: {f}')
        copied += 1

print(f'\nCopied {copied} files to {DST}')
!ls -lh {DST}/

## 6. Resume Training (if interrupted)

If Colab disconnected, re-run cells 1-2 (mount + clone), then run this cell.
It resumes from the best checkpoint saved in the results folder.

In [ ]:
# ── Match your initial config settings ──
COLAB_BATCH_SIZE = 64
BASE_LR = 0.0003
BASE_BS = 16
COLAB_LR = BASE_LR * (COLAB_BATCH_SIZE / BASE_BS)

# First, restore checkpoint from Drive if results folder is empty
SRC_DRIVE = '/content/drive/MyDrive/Ocean Inpainting/training_results/fwd_divfree_equalized'
DST_LOCAL = f'{REPO_DIR}/experiments/01_noise_strategy/fwd_divfree_equalized/results'
os.makedirs(DST_LOCAL, exist_ok=True)

if os.path.exists(SRC_DRIVE):
    for f in os.listdir(SRC_DRIVE):
        if f.endswith('.pt'):
            dst_path = os.path.join(DST_LOCAL, f)
            if not os.path.exists(dst_path):
                shutil.copy2(os.path.join(SRC_DRIVE, f), dst_path)
                print(f'  Restored: {f}')

# Find best checkpoint
ckpt = None
for f in os.listdir(DST_LOCAL):
    if 'best_checkpoint' in f and f.endswith('.pt'):
        ckpt = os.path.join(DST_LOCAL, f)
        break

if ckpt:
    print(f'Resuming from: {ckpt}')
    # Create a temporary resume config with all improvements
    resume_yaml = os.path.join(DST_LOCAL, 'resume_config.yaml')
    with open(resume_yaml, 'w') as f:
        f.write(f"""# Auto-generated resume config with training improvements
model_name: fwd_diff_eq_divfree_eps_t250
noise_function: fwd_diff_eq_divfree
unet_type: standard
prediction_target: eps
mask_xt: false
p_uncond: 0.0
batch_size: {COLAB_BATCH_SIZE}
lr: {COLAB_LR}
max_grad_norm: 1.0

# Training improvements
weight_decay: 0.0001
lr_schedule: cosine
warmup_epochs: 10
use_ema: true
ema_decay: 0.9999
augment: false

# Resume settings
retrain_mode: true
model_to_retrain: {ckpt}
reset_best: false
""")
    !PYTHONPATH=. python experiments/run_experiment.py {resume_yaml}
else:
    print('No checkpoint found — starting fresh')
    COLAB_CFG = f'{REPO_DIR}/experiments/01_noise_strategy/fwd_divfree_equalized/colab_config.yaml'
    !PYTHONPATH=. python experiments/run_experiment.py {COLAB_CFG}

## 7. Download Best Checkpoint Locally

After training completes, download the checkpoint to use on your local machine.

In [ ]:
from google.colab import files

RESULTS = f'{REPO_DIR}/experiments/01_noise_strategy/fwd_divfree_equalized/results'
for f in os.listdir(RESULTS):
    if 'best_checkpoint' in f and f.endswith('.pt'):
        print(f'Downloading: {f}')
        files.download(os.path.join(RESULTS, f))
        break